In [87]:
import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Merge, Embedding, Convolution1D,Dropout
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from scipy import signal

from features import *
from helper import *


In [25]:

code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [3]:
import pickle
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [54]:
fs = 16e3
f, t, Sxx = signal.spectrogram(data2[1000]['signal'], fs,nperseg=400)
Sxx, _ = pad_sequence_into_array(Sxx, maxlen=300)
Sxx.shape

(201, 300)

In [55]:
x_train_speech = []
fs = 16e3
counter = 0
for ses_mod in data2:
    x_speech = ses_mod['signal']
    f, t, Sxx = signal.spectrogram(x_speech, fs, nperseg=400)
    Sxx, _ = pad_sequence_into_array(Sxx, maxlen=300)
    x_train_speech.append(Sxx)
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_speech = np.array(x_train_speech)
x_train_speech.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 201, 300)

In [56]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4936, 4)

In [61]:
model = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Convolution1D(256, 3, border_mode='same', input_shape=(201, 300)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu')) 
model.add(Dropout(0.2))
model.add(Dense(4))
model.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 201, 256)          230656    
_________________________________________________________________
dropout_43 (Dropout)         (None, 201, 256)          0         
_________________________________________________________________
activation_43 (Activation)   (None, 201, 256)          0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 201, 128)          98432     
_________________________________________________________________
dropout_44 (Dropout)         (None, 201, 128)          0         
_________________________________________________________________
activation_44 (Activation)   (None, 201, 128)          0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 201, 64)           24640     
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same", input_shape=(201, 300))`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
  if sys.path[0] == '':


In [62]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=25, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/25
3948/3948 [==============================] - 2s - loss: 1.5278 - acc: 0.3655 - val_loss: 1.2893 - val_acc: 0.3927
Epoch 2/25
3948/3948 [==============================] - 2s - loss: 1.3808 - acc: 0.4151 - val_loss: 1.2075 - val_acc: 0.4383
Epoch 3/25
3948/3948 [==============================] - 2s - loss: 1.2551 - acc: 0.4260 - val_loss: 1.1657 - val_acc: 0.4899
Epoch 4/25
3948/3948 [==============================] - 2s - loss: 1.2106 - acc: 0.4491 - val_loss: 1.1853 - val_acc: 0.4514
Epoch 5/25
3948/3948 [==============================] - 2s - loss: 1.1702 - acc: 0.4580 - val_loss: 1.2019 - val_acc: 0.4747
Epoch 6/25
3948/3948 [==============================] - 2s - loss: 1.1756 - acc: 0.4633 - val_loss: 1.1700 - val_acc: 0.4909
Epoch 7/25
3948/3948 [==============================] - 2s - loss: 1.1674 - acc: 0.4770 - val_loss: 1.1655 - val_acc: 0.5051
Epoch 8/25
3948/3948 [==============================] - 2s - loss: 1.1402 - ac

In [82]:
def lstm_model(optimizer='Adadelta'):
    model = Sequential()
    model.add(LSTM(256, return_sequences=True, input_shape=(201, 300)))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [88]:
model = lstm_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 201, 256)          570368    
_________________________________________________________________
lstm_10 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_31 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_71 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 4)                 2052      
_________________________________________________________________
activation_72 (Activation)   (None, 4)                 0         
Total params: 1,229,316
Trainable params: 1,229,316
Non-trainable params: 0
_________________________________________________________________


In [89]:
hist = model.fit(x_train_speech, Y, 
                 batch_size=100, nb_epoch=40, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/40
3948/3948 [==============================] - 27s - loss: 1.3685 - acc: 0.3478 - val_loss: 1.3650 - val_acc: 0.3694
Epoch 2/40
3948/3948 [==============================] - 27s - loss: 1.2988 - acc: 0.3713 - val_loss: 1.2931 - val_acc: 0.4109
Epoch 3/40
3948/3948 [==============================] - 26s - loss: 1.2345 - acc: 0.4119 - val_loss: 1.2985 - val_acc: 0.3431
Epoch 4/40
3948/3948 [==============================] - 26s - loss: 1.2043 - acc: 0.4392 - val_loss: 1.2459 - val_acc: 0.4241
Epoch 5/40
3948/3948 [==============================] - 27s - loss: 1.1888 - acc: 0.4521 - val_loss: 1.2853 - val_acc: 0.3765
Epoch 6/40
3948/3948 [==============================] - 26s - loss: 1.1692 - acc: 0.4547 - val_loss: 1.2403 - val_acc: 0.4251
Epoch 7/40
3948/3948 [==============================] - 26s - loss: 1.1447 - acc: 0.4754 - val_loss: 1.2963 - val_acc: 0.3957
Epoch 8/40
3948/3948 [==============================] - 26s - loss: 1.1

KeyboardInterrupt: 

In [90]:
def calculate_features(frames, freq, options):
    window_sec = 0.2
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [91]:
x_train_feat = []

counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calculate_features(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    x_train_feat.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_feat = np.array(x_train_feat)
x_train_feat.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 100, 34)

In [92]:
def linear_model_combined(optimizer='Adam'):
    modela = Sequential()
    modela.add(Flatten(input_shape=(100, 34)))
    modela.add(Dense(1024))
    modela.add(Activation('relu'))
    modela.add(Dense(256))
    
    modelb = Sequential()
    #model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
    modelb.add(Convolution1D(256, 3, border_mode='same', input_shape=(201, 300)))
    modelb.add(Dropout(0.2))
    modelb.add(Activation('relu'))
    modelb.add(Convolution1D(128, 3, border_mode='same'))
    modelb.add(Dropout(0.2))
    modelb.add(Activation('relu'))
    modelb.add(Convolution1D(64, 3, border_mode='same'))
    modelb.add(Dropout(0.2))
    modelb.add(Activation('relu'))
    modelb.add(Convolution1D(32, 3, border_mode='same'))
    modelb.add(Dropout(0.2))
    modelb.add(Activation('relu'))
    modelb.add(Flatten())
    modelb.add(Dense(256))
    
    model_combined = Sequential()
    model_combined.add(Merge([modela, modelb], mode='concat'))
    model_combined.add(Activation('relu'))
    
    model_combined.add(Dense(4))
    model_combined.add(Activation('softmax'))

    model_combined.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model_combined

In [93]:
model = linear_model_combined()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_1 (Merge)              (None, 512)               0         
_________________________________________________________________
activation_78 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 4)                 2052      
_________________________________________________________________
activation_79 (Activation)   (None, 4)                 0         
Total params: 5,753,828
Trainable params: 5,753,828
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same", input_shape=(201, 300))`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [94]:
hist = model.fit([x_train_feat, x_train_speech], Y, 
                 batch_size=100, nb_epoch=25, verbose=1, shuffle = True, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/25
3948/3948 [==============================] - 3s - loss: 3.1072 - acc: 0.3782 - val_loss: 1.3515 - val_acc: 0.4474
Epoch 2/25
3948/3948 [==============================] - 2s - loss: 1.2129 - acc: 0.4580 - val_loss: 1.2241 - val_acc: 0.4757
Epoch 3/25
3948/3948 [==============================] - 2s - loss: 1.1117 - acc: 0.5182 - val_loss: 1.2740 - val_acc: 0.4443
Epoch 4/25
3948/3948 [==============================] - 2s - loss: 1.0707 - acc: 0.5420 - val_loss: 1.3084 - val_acc: 0.4575
Epoch 5/25
3948/3948 [==============================] - 2s - loss: 1.0295 - acc: 0.5605 - val_loss: 1.3696 - val_acc: 0.4555
Epoch 6/25
3948/3948 [==============================] - 2s - loss: 0.9619 - acc: 0.5864 - val_loss: 1.3387 - val_acc: 0.4980
Epoch 7/25
3948/3948 [==============================] - 2s - loss: 0.9426 - acc: 0.5899 - val_loss: 1.3813 - val_acc: 0.4798
Epoch 8/25
3948/3948 [==============================] - 2s - loss: 0.8998 - ac